In [1]:
input_file = "14_input.txt"

In [2]:
class Cave:
    def __init__(self, xlim, ylim, start=[500, 0], lines=None, floor=False):
        self.start = start
        self.xmin, self.xmax = xlim
        self.ymin, self.ymax = ylim
        self.ymin = min(self.ymin, 0)
        if floor:
            self.ymax += 2
            h = self.ymax - self.ymin
            self.xmin = min(self.xmin, start[0]-h)
            self.xmax = max(self.xmax, start[0]+h)
        self.grid = [[0] * (1 + self.xmax - self.xmin) for _ in range(1 + self.ymax - self.ymin)]
        if lines is not None:
            for line in lines:
                self.add_line(line)
        if floor:
            self.grid[-1] = [1 for _ in range(self.xmin, self.xmax+1)] 


    def add_line(self, line):
        for k, start in enumerate(line[:-1]):
            end = line[k+1]
            if start[0] == end[0]: #vertical line
                yrange = range(min(start[1], end[1]), max(start[1], end[1]) + 1)
                for y in yrange:
                    self.grid[y-self.ymin][start[0]-self.xmin] = 1
            elif start[1] == end[1]: #horizontal line
                xrange = range(min(start[0], end[0]), max(start[0], end[0]) + 1)
                for x in xrange:
                    self.grid[start[1]-self.ymin][x-self.xmin] = 1
            else:
                raise ValueError("Lines must be either vertical or horizontal")
    
    
    def fill_sand(self, maxgrains=10**5):
        full = False
        c = 0
        while not(full) and c<maxgrains:
            sx, sy = self.start
            rest = False
            while not(rest or full):
                if sy == self.ymax:
                    full = True
                elif self.grid[sy+1-self.ymin][sx-self.xmin] == 0:
                    sy += 1
                elif self.grid[sy+1-self.ymin][sx-1-self.xmin] == 0: #what if reached limit?
                    sy += 1
                    sx -= 1
                elif self.grid[sy+1-self.ymin][sx+1-self.xmin] == 0:
                    sy += 1
                    sx += 1
                else:
                    rest = True
                    self.grid[sy-self.ymin][sx-self.xmin] = 2
                    c += 1
                    if [sx, sy] == self.start:
                        full = True
        self.atrest = c

    
    def print(self, pixels=['.', '#', 'o'], startpix='+', line_numbers=False):
        for k, row in enumerate(self.grid):
            screenline = [pixels[x] for x in row]
            if k + self.ymin == self.start[1]:
                screenline[self.start[0]-self.xmin] = startpix
            if line_numbers:
                print(f"{k:03d} ", end="")
            print(''.join(screenline))

In [3]:
xlim = [500, 500]
ylim = [0, 0]

lines = []

with open(input_file) as f:
    for line in f:
        newline = [[int(y) for y in x.split(',')] for x in line.rstrip().split(" -> ")]
        lines.append(newline)


x_all = [p[0] for l in lines for p in l]
y_all = [p[1] for l in lines for p in l]

xlim = [min(x_all), max(x_all)]
ylim = [min(min(y_all),0), max(y_all)]

In [4]:
cave = Cave(xlim, ylim, lines=lines)

cave.fill_sand()
print(cave.atrest)

1068


In [5]:
len(cave.grid)

172

In [6]:
floored_cave = Cave(xlim, ylim, lines=lines, floor=True)

floored_cave.fill_sand()
print(floored_cave.atrest)

27936
